In [5]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pylab as plt
import cv2

In [6]:
train_dir="./tensorflow/cats_and_dogs_train/"
test_dir="./tensorflow/cats_and_dogs_test/"

In [7]:
image_size=128
channel=3
pixel_depth=255.0


train_and_valid_dogs=1750
train_and_valid_cats=1750
train_and_valid_all=3500
train_size=3000
valid=500
test_size=500


train_images=[train_dir+ i for i  in os.listdir(train_dir)]
train_dog=[train_dir+i for i in os.listdir(train_dir) if "dog" in i]
train_cat=[train_dir+i for i in os.listdir(train_dir) if "cat" in i]
test_images=[test_dir+i for i in os.listdir(test_dir)]

train_images=train_dog[:train_and_valid_dogs]+train_cat[:train_and_valid_cats]
train_labels=np.array((["dogs"]*train_and_valid_dogs)+(["cats"]*train_and_valid_cats))
test_images=test_images[:test_size]
test_labels=np.array(["unknownclass"]*test_size)

# 이미지 전처리

In [8]:
def read_image(path):
    img=cv2.imread(path,cv2.IMREAD_COLOR)
    if(img.shape[0]>=img.shape[1]):# 높이가 넓이보다 긴경우
        resizeto=(image_size,int(round(image_size*(float(img.shape[1])/img.shape[0]))))
    else:
        resizeto=(int(round(image_size*(float(img.shape[0])/img.shape[1]))),image_size)
        
    img2 = cv2.resize(img, (resizeto[1], resizeto[0]), interpolation=cv2.INTER_CUBIC)
    img3 = cv2.copyMakeBorder(img2, 0, image_size - img2.shape[0], 0, image_size - img2.shape[1], cv2.BORDER_CONSTANT, 0)
    
    return img3[:,:,::-1]

def prep_data(images):
    count = len(images)
    data =np.ndarray((count,128,128,channel),dtype=np.float32)

    for i, image_file in enumerate(images):
        image = read_image(image_file);
        image_data = np.array (image, dtype=np.float32);
        image_data[:,:,0] = (image_data[:,:,0].astype(float) - pixel_depth / 2) / pixel_depth
        image_data[:,:,1] = (image_data[:,:,1].astype(float) - pixel_depth / 2) / pixel_depth
        image_data[:,:,2] = (image_data[:,:,2].astype(float) - pixel_depth / 2) / pixel_depth
        
        data[i] = image_data
        if i%250 == 0: 
            print('Processed {} of {}'.format(i, count))    
    return data


In [9]:
train_normalization=prep_data(train_images)
test_noramlization=prep_data(test_images)

Processed 0 of 3500
Processed 250 of 3500
Processed 500 of 3500
Processed 750 of 3500
Processed 1000 of 3500
Processed 1250 of 3500
Processed 1500 of 3500
Processed 1750 of 3500
Processed 2000 of 3500
Processed 2250 of 3500
Processed 2500 of 3500
Processed 2750 of 3500
Processed 3000 of 3500
Processed 3250 of 3500
Processed 0 of 500
Processed 250 of 500


In [10]:
np.random.seed (133)
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

train_dataset_rand, train_labels_rand = randomize(train_normalization, train_labels)
test_dataset, test_labels = randomize(test_noramlization, test_labels)


valid_dataset = train_dataset_rand[:valid,:,:,:]
valid_labels =   train_labels_rand[:valid]
train_dataset = train_dataset_rand[valid:,:,:,:]
train_labels  = train_labels_rand[valid:]
print ('Training', train_dataset.shape, train_labels.shape)
print ('Validation', valid_dataset.shape, valid_labels.shape)
print ('Test', test_dataset.shape, test_labels.shape)

Training (3000, 128, 128, 3) (3000,)
Validation (500, 128, 128, 3) (500,)
Test (500, 128, 128, 3) (500,)


# cnn 

In [11]:
image_size=128
num_labels=2
num_channels=3

def reformat(dataset,labels):
    dataset=dataset.reshape((-1,128,128,num_channels)).astype(np.float32)
    labels=(labels=="cats").astype(np.float32)
    labels=(np.arange(num_labels)==labels[:,None]).astype(np.float32)
    return dataset , labels

In [12]:
train_dataset,train_labels=reformat(train_dataset,train_labels)
test_dataset,test_labels=reformat(test_dataset,test_labels)
valid_dataset,valid_labels=reformat(valid_dataset,valid_labels)

In [19]:

batch_size=30
patch_size=5
depth=16
num_hidden=64

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    
    
    kernel_conv1=tf.Variable(tf.truncated_normal([3,3,3,32],dtype=tf.float32,stddev=1e-1),name="weights_conv1")
    biases_conv1=tf.Variable(tf.constant(0.0,shape=[32],dtype=tf.float32),trainable=True,name="biases_conv1")
    
    
    
    kernel_conv2=tf.Variable(tf.truncated_normal([3,3,32,32],dtype=tf.float32,stddev=1e-1),name="weights_conv2")
    biases_conv2=tf.Variable(tf.constant(0.0,shape=[32],dtype=tf.float32),trainable=True,name="biases_conv2")
        
    
    kernel_conv3=tf.Variable(tf.truncated_normal([3,3,32,64],dtype=tf.float32,stddev=1e-1),name="weights_conv3")
    biases_conv3=tf.Variable(tf.constant(0.0,shape=[64],dtype=tf.float32),trainable=True,name="biases_conv3")
    
    fc1w=tf.Variable(tf.truncated_normal([16*16*64,64],dtype=tf.float32,stddev=1e-1),name="weights")
    fc1b=tf.Variable(tf.constant(1.0,shape=[64],dtype=tf.float32),trainable=True,name="biases")
    
    fc2w=tf.Variable(tf.truncated_normal([64,2],dtype=tf.float32,stddev=1e-1),name="weights")
    fc2b=tf.Variable(tf.constant(1.0,shape=[2],dtype=tf.float32),trainable=True,name="biases")
    
    
    def model(data):
        parameters=[]
        with tf.name_scope("conv1_1") as scope:
            conv=tf.nn.conv2d(data,kernel_conv1,[1,1,1,1],padding="SAME")
            out=tf.nn.bias_add(conv,biases_conv1)
            conv1_1=tf.nn.relu(out,name=scope)
            parameters+=[kernel_conv1,biases_conv1]

            pool1=tf.nn.max_pool(conv1_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name="pool1")

        
        with tf.name_scope("conv2_1") as scope:
            conv=tf.nn.conv2d(pool1,kernel_conv2,[1,1,1,1],padding="SAME")
            out=tf.nn.bias_add(conv,biases_conv2)
            conv2_1=tf.nn.relu(out,name=scope)
            parameters+=[kernel_conv2,biases_conv2]

            pool2=tf.nn.max_pool(conv2_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name="pool2")
    
        with tf.name_scope("conv3_1") as scope:
            conv=tf.nn.conv2d(pool2,kernel_conv3,[1,1,1,1],padding="SAME")
            out=tf.nn.bias_add(conv,biases_conv3)
            conv3_1=tf.nn.relu(out,name=scope)
            parameters+=[kernel_conv3,biases_conv3]

            pool3=tf.nn.max_pool(conv3_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name="pool3")

        
        
        with tf.name_scope("fc1") as scope:
            shape=int(np.prod(pool3.get_shape()[1:]))
            pool3_flat=tf.reshape(pool3,[-1,shape])
            fc1l=tf.nn.bias_add(tf.matmul(pool3_flat,fc1w),fc1b)
            fc1=tf.nn.relu(fc1l)
            parameters+=[fc1w,fc1b]
        
        
        with tf.name_scope("fc2") as scope:
            fc2l=tf.nn.bias_add(tf.matmul(fc1,fc2w),fc2b)
            parameters+=[fc2w,fc2b]
            
        return fc2l
        
         
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    optimizer = tf.train.RMSPropOptimizer(0.0001).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
    
    


In [26]:
def accuracy(predictions, labels):
   return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

num_steps = 10001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print ("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 0.936668
Minibatch accuracy: 56.7%
Validation accuracy: 45.6%
Minibatch loss at step 50 : 0.808521
Minibatch accuracy: 26.7%
Validation accuracy: 48.2%
Minibatch loss at step 100 : 0.733869
Minibatch accuracy: 50.0%
Validation accuracy: 55.4%
Minibatch loss at step 150 : 1.00538
Minibatch accuracy: 36.7%
Validation accuracy: 48.4%
Minibatch loss at step 200 : 0.621105
Minibatch accuracy: 60.0%
Validation accuracy: 57.2%
Minibatch loss at step 250 : 0.603547
Minibatch accuracy: 66.7%
Validation accuracy: 58.4%
Minibatch loss at step 300 : 0.530795
Minibatch accuracy: 83.3%
Validation accuracy: 56.8%
Minibatch loss at step 350 : 0.561747
Minibatch accuracy: 70.0%
Validation accuracy: 58.2%
Minibatch loss at step 400 : 0.493519
Minibatch accuracy: 76.7%
Validation accuracy: 58.6%
Minibatch loss at step 450 : 0.57402
Minibatch accuracy: 70.0%
Validation accuracy: 60.6%
Minibatch

### reference:kagggle.com/kbhits/tensorflow-starter-kit-fixed